In [2]:
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever(
    lang="ko",
    doc_content_chars_max=100,
    top_k_results=3
)

In [10]:
result = retriever.invoke("러우 전쟁")

In [14]:
result

[Document(metadata={'title': '러시아-우크라이나 전쟁', 'summary': '러시아-우크라이나 전쟁(러시아어: российско-украинская война, 우크라이나어: російсько-українська війна) 또는 러우 전쟁은 2014년 2월부터 시작된 분쟁으로, 우크라이나와 침략국 러시아 및 우크라이나 내 친러 분리주의 세력 간의 분쟁을 뜻한다. 여러 국가가 우크라이나에게 다양한 수준의 지원을 제공하였으며, 벨라루스는 2022년 러시아의 우크라이나 전면 침공 당시 벨라루스 영토 내 러시아군의 통행을 제공하였다. 2014년 우크라이나 혁명 직후 러시아가 적대 행위를 시작하였으며 이는 대부분 크림반도와 돈바스의 정치적 지위 문제에 집중되었는데 돈바스의 경우에는 국제적으로 우크라이나의 영토로 인정받고 있다. 우크라이나 영토 침공 행위는 러시아의 크림반도 합병으로 절정에 달했고, 그로부터 얼마 지나지 않아 러시아의 지원을 받는 분리주의 세력과 우크라이나 사이 돈바스 전쟁이 발발하였다. 그 외에도 분쟁 시작 후 8년 동안 해상 충돌, 사이버전, 다양한 정치적 긴장 고조 등 여러 사건이 발생하였다. 2021년 내내 우크라이나 영토 주변 러시아군의 증강으로 양국 간 긴장이 고조되었으며, 2022년 2월 24일 러시아의 우크라이나 침공으로 분쟁이 전면전으로 격화되었다.\n2014년 2월 2일, 당시 우크라이나의 대통령이었던 빅토르 야누코비치가 유럽 연합-우크라이나 공동체 협정 협상을 거부하고 유라시아 경제 연합과 더욱 긴밀히 연계하기로 결정하자 발생한 유로마이단 시위와 이후 일어난 혁명으로 쫓겨났다. 야누코비치가 러시아로 망명한 직후 동부와 남부 우크라이나에서는 친러 봉기가 발발하였다. 같은 시기 아무런 표식이 없는 러시아군이 우크라이나의 크림반도로 이동하여 주요 전략지대와 인프라를 장악하고, 2014년 2월 27일에는 크림의 의회 건물도 점령하였다. 3월 16일에는 러시아로 합병하자는 크림 주민투표를 열고 주민투표가 열린지 이틀 

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [5]:
template = """
당신은 주어진 맥락(context)만을 사용하여 질문에 답변하는 AI 어시스턴트입니다.
절대 당신의 사전 지식을 사용하지 마세요. 맥락에서 답을 찾을 수 없다면, "모르겠습니다"라고 답변하세요.
# 맥락 (Context):
{context}
# 질문 (Question):
{question}
"""
prompt = ChatPromptTemplate.from_template(template) 

In [6]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path="../../.env")
api_key = os.getenv("API_KEY")

In [8]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-4.1-2025-04-14', temperature=0, api_key=api_key)
output_parser = StrOutputParser()

In [11]:
tmp = []
for words in result:
    tmp.append(words.metadata['summary'])
texts = "\n".join(tmp)
texts = "\n".join([words.metadata['summary'] for words in result])
def format_docs(docs):
    return "\n".join([words.metadata['summary'] for words in docs])

chain  = {"context"  : retriever | format_docs, "question": RunnablePassthrough()} | \
 prompt | model | output_parser

In [ ]:
response = chain.invoke("러우 전쟁과 베트남 전쟁의 비슷한 부분 알려줘")

In [13]:
response

'모르겠습니다.'

In [ ]:
pip install unstructured

In [ ]:
pip install "unstructured[docx]"

In [19]:
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = UnstructuredFileLoader("./data/ADsP - 정리.docx")
documents = loader.load()

/tmp/ipykernel_24831/2781564609.py:6: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader("./data/ADsP - 정리.docx")


In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) 
docs = text_splitter.split_documents(documents)

In [ ]:
pip install chromadb

In [29]:
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
embeddings = OpenAIEmbeddings(api_key=api_key)
db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings
)
db.similarity_search("정규분포")

/tmp/ipykernel_24831/3151611992.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


[Document(metadata={'source': './data/ADsP - 정리.docx'}, page_content="8) 포아송분포: 단위 시간 내 발생할 수 있는 사건의 발생 횟수에 대한 분포\n\n예) 하루동안 발생하는 출생자 수, 한 시간 동안 사무실에 걸려온 전화의 수\n\n☞ '베포항항하'\n\n(2) 연속 확률분포\n\n값을 셀 수 없는 분포, 확률밀도함수로 표현\n\n1) 정규분포: 우리가 일상생활에서 흔히 보는 확률변수의 평균 분포를 근사한 분포 (Z검정 활용)\n\n예) 사람들의 키 혹은 IQ 점수의 분포, 시험 성적의 분포\n\n2) t분포: 정규분포와 유사하지만, 꼬리 부분이 더 두껍고 긴 분포\n\n(T검정 활용) 표본이 30개 보다 작은 집단에 대한 평균 검정\n\n3) 카이제곱분포: 독립적인 정규분포를 따르는 변수들의 제곱합으로 구성된 분포\n\n(카이제곱 검정 활용) 두 집단의 동질성 검정, 단일 집단의 모분산 검정\n\n4) F분포: 두 개의 서로 다른 카이제곱 분포의 비율\n\n(F검정 활용) 두 집단의 분산 동질성 검정\n\n확률분포의 기댓값\n\n확률변수 X의 f(x) 확률분포의 대한 기댓값(E(X))\n\n1) 이산 확률변수: E(X)= ∑ xf(x)\n\n\n\n2) 연속적 확률변수: E(X)=∫ xf(x)\n\n중심극한정리\n\n임의의 모집단으로부터 추출된 표본분포는 표본크기가 충분히 크면(30개 이상) 정규분포\n\n모집단의 분포에 상관없이 표본분포가 정규분포를 이룸\n\n표본평균의 표본분포\n\n(1) 표본평균의 표본분포의 평균: \n\n(2) 표본평균의 표본분포의 분산: \n\n(3) 표본평균의 표준화: \n\n(: 모집단의 평균, : 모집단의 표준편차, : 표본평균, : 표본의 크기)\n\n점추정\n\n모집단이 특정한 값으로 추정하며, 추정량(Estimator)으로 모수를 추정\n\n(1) 추정량의 조건\n\n1) 불편성(Unbiasedness) : 추정량의 기댓값이 실제 모수와 같음 (편향이 0이 되는 

In [30]:
db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

rt = db.similarity_search("정규분포는 무엇인가?")

retriever = db.as_retriever()
def format_docs(docs):
    return "\n".join([words.page_content for words in docs])

chain_db  = {"context"  : retriever | format_docs, "question": RunnablePassthrough()} | \
 prompt | model | output_parser

result_doc = chain_db.invoke("정규분포에 대한 시험문제")

template = """
당신은 주어진 맥락(context)만을 사용하여 질문에 답변하는 AI 어시스턴트입니다.
절대 당신의 사전 지식을 사용하지 마세요. 제공된 맥락에서 아래 제시된 예시처럼 최소 3문제 이상 생성하세요.

# 예시
1. 다음중 데이터의 특성이 다른 하나는?
① 풍량
② 강수량
③ 기상특보
④ 습도

# 맥락 (Context):
{context}

# 질문 (Question):
{question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain_db  = {"context"  : retriever | format_docs, "question": RunnablePassthrough()} | \
 prompt | model | output_parser
result_doc = chain_db.invoke("정규분포에 대한 시험문제")


In [31]:
result_doc

'1. 다음 중 정규분포의 예시로 가장 적절한 것은?\n① 하루 동안 발생하는 출생자 수  \n② 한 시간 동안 사무실에 걸려온 전화의 수  \n③ 사람들의 키 분포  \n④ 두 집단의 분산 동질성 검정  \n\n2. 정규분포에 대한 설명으로 옳지 않은 것은?\n① 일상생활에서 흔히 볼 수 있는 확률변수의 평균 분포를 근사한다.  \n② 확률밀도함수로 표현되는 연속 확률분포이다.  \n③ 표본이 30개보다 작은 집단의 평균 검정에 주로 사용된다.  \n④ Z검정에 활용된다.  \n\n3. 다음 중 정규분포와 가장 관련이 깊은 통계적 검정 방법은?\n① T검정  \n② Z검정  \n③ 카이제곱 검정  \n④ F검정  \n\n4. 중심극한정리에 따르면, 표본의 크기가 충분히 크면 표본평균의 분포는 어떤 분포를 따르는가?\n① 포아송분포  \n② 정규분포  \n③ 카이제곱분포  \n④ F분포  '